In [ ]:
"""
Text generation using a Recurrent Neural Network (LSTM).
"""

import argparse
import os
import random
import time

import numpy as np
import tensorflow as tf

import pickle
from tensorflow.python.framework import graph_util # used for exporting the graph

class ModelNetwork:
    """
    RNN with num_layers LSTM layers and a fully-connected output layer
    The network allows for a dynamic number of iterations, depending on the
    inputs it receives.

       out   (fc layer; out_size)
        ^
       lstm
        ^
       lstm  (lstm size)
        ^
        in   (in_size)
    """
    def __init__(self, in_size, lstm_size, num_layers, out_size,output_tensor, session,
                 learning_rate=0.003, name="rnn"):
        self.scope = name
        self.in_size = in_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.out_size = out_size
        self.output_tensor = output_tensor
        self.session = session
        self.learning_rate = tf.constant(learning_rate)
        # Last state of LSTM, used when running the network in TEST mode
        self.lstm_last_state = np.zeros(
            (self.num_layers * 2 * self.lstm_size,)
        )
        with tf.variable_scope(self.scope):
            # (batch_size, timesteps, in_size)
            self.xinput = tf.placeholder(
                tf.float32,
                shape=(None, None, self.in_size),
                name="xinput"
            )
            self.lstm_init_value = tf.placeholder(
                tf.float32,
                shape=(None, self.num_layers * 2 * self.lstm_size),
                name="lstm_init_value"
            )
            # LSTM
            self.lstm_cells = [
                tf.contrib.rnn.BasicLSTMCell(
                    self.lstm_size,
                    forget_bias=1.0,
                    state_is_tuple=False
                ) for i in range(self.num_layers)
            ]
            self.lstm = tf.contrib.rnn.MultiRNNCell(
                self.lstm_cells,
                state_is_tuple=False
            )
            # Iteratively compute output of recurrent network
            outputs, self.lstm_new_state = tf.nn.dynamic_rnn(
                self.lstm,
                self.xinput,
                initial_state=self.lstm_init_value,
                dtype=tf.float32
            )
            # Linear activation (FC layer on top of the LSTM net)
            self.rnn_out_W = tf.Variable(
                tf.random_normal(
                    (self.lstm_size, self.out_size),
                    stddev=0.01
                )
            )
            self.rnn_out_B = tf.Variable(
                tf.random_normal(
                    (self.out_size,), stddev=0.01
                )
            )
            outputs_reshaped = tf.reshape(outputs, [-1, self.lstm_size])
            
            #network_output_1 = f.matmul(outputs_reshaped,self.rnn_out_W  )+self.rnn_out_B
            
            network_output_1 = tf.matmul(outputs_reshaped,self.rnn_out_W  )
            network_output = tf.add(network_output_1, self.rnn_out_B,  name ="outputXYZ")  
            
#             network_output_1 =  tf.add(network_output,  name =self.output_tensor)                      
                                      
            
            #network_output = tf.identity(network_output_1, self.output_tensor)
            '''network_output = tf.add(tf.matmul(
                outputs_reshaped,
                self.rnn_out_W
            ),self.rnn_out_B,name = output_tensor)'''
                
            batch_time_shape = tf.shape(outputs)
            self.final_outputs = tf.reshape(
                tf.nn.softmax(network_output),
                (batch_time_shape[0], batch_time_shape[1], self.out_size),       
            )
            #,name = output_tensor
            # Training: provide target outputs for supervised training.
            self.y_batch = tf.placeholder(
                tf.float32,
                (None, None, self.out_size)
            )
            y_batch_long = tf.reshape(self.y_batch, [-1, self.out_size])
            self.cost = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(
                    logits=network_output,
                    labels=y_batch_long
                )
            )
            self.train_op = tf.train.RMSPropOptimizer(
                self.learning_rate,
                0.9
            ).minimize(self.cost)

    # Input: X is a single element, not a list!
    def run_step(self, x, init_zero_state=True):
        # Reset the initial state of the network.
        if init_zero_state:
            init_value = np.zeros((self.num_layers * 2 * self.lstm_size,))
        else:
            init_value = self.lstm_last_state
        out, next_lstm_state = self.session.run(
            [self.final_outputs, self.lstm_new_state],
            feed_dict={
                self.xinput: [x],
                self.lstm_init_value: [init_value]
            }
        )
        self.lstm_last_state = next_lstm_state[0]
        return out[0][0]

    # xbatch must be (batch_size, timesteps, input_size)
    # ybatch must be (batch_size, timesteps, output_size)
    def train_batch(self, xbatch, ybatch):
        init_value = np.zeros(
            (xbatch.shape[0], self.num_layers * 2 * self.lstm_size)
        )
        cost, _ = self.session.run(
            [self.cost, self.train_op],
            feed_dict={
                self.xinput: xbatch,
                self.y_batch: ybatch,
                self.lstm_init_value: init_value
            }
        )
        return cost


def embed_to_vocab(data_, vocab):
    """
    Embed string to character-arrays -- it generates an array len(data)
    x len(vocab).

    Vocab is a list of elements.
    """
    data = np.zeros((len(data_), len(vocab)))
    cnt = 0
    for s in data_:
        v = [0.0] * len(vocab)
        v[vocab.index(s)] = 1.0
        data[cnt, :] = v
        cnt += 1
    return data


def decode_embed(array, vocab):
    return vocab[array.index(1)]


def load_data(input):
    # Load the data
    data_ = ""
    with open(input, 'r') as f:
        data_ += f.read()
    data_ = data_.lower()
    # Convert to 1-hot coding
    vocab = sorted(list(set(data_)))
    data = embed_to_vocab(data_, vocab)
    return data, vocab


def check_restore_parameters(sess, saver):
    """ Restore the previously trained parameters if there are any. """
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('saved/checkpoint'))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)



In [ ]:
#below cell
#args = parser.parse_args()
mode="train"

TEST_PREFIX = "The " # Prefix to prompt the network in test mode

#if args.ckpt_file:
    #ckpt_file = args.ckpt_file

# Load the data
#data, vocab = load_data('tokenized_part_instructions_train.txt')
data, vocab = load_data('dataset/tokenized_recipe_train1.txt')
#data, vocab = load_data('shakespeare.txt')

In [ ]:

###
save_path = "./model/"
in_size = out_size = len(vocab)
lstm_size = 256  # 128
num_layers = 2
batch_size = 32 #64  # 128
time_steps = 500  # 50

NUM_TRAIN_BATCHES = 10000

# Number of test characters of text to generate after training the network
LEN_TEST_TEXT = 300


output_tensor = "output"
# Initialize the network
GPU_USE = '/gpu:0'
config = tf.ConfigProto()
config.log_device_placement=True
config.allow_soft_placement=True
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction=0.6
config.gpu_options.allocator_type = 'BFC'

scope = "char_rnn_network"
with tf.Session(config=config) as sess:
    net = ModelNetwork(
        in_size=in_size,
        lstm_size=lstm_size,
        num_layers=num_layers,
        out_size=out_size,
        output_tensor=output_tensor,
        session=sess,
        learning_rate=0.001,
        name=scope
    )

    #print(net.output_tensor)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    #saver = tf.train.Saver()

    # 1) TRAIN THE NETWORK
    if mode == "train":
        check_restore_parameters(sess, saver)
        last_time = time.time()
        batch = np.zeros((batch_size, time_steps, in_size))
        batch_y = np.zeros((batch_size, time_steps, in_size))
        possible_batch_ids = range(data.shape[0] - time_steps - 1)

        for i in range(NUM_TRAIN_BATCHES):
            # Sample time_steps consecutive samples from the dataset text file
            batch_id = random.sample(possible_batch_ids, batch_size)

            for j in range(time_steps):
                ind1 = [k + j for k in batch_id]
                ind2 = [k + j + 1 for k in batch_id]

                batch[:, j, :] = data[ind1, :]
                batch_y[:, j, :] = data[ind2, :]

            cst = net.train_batch(batch, batch_y)

            if (i % 100) == 0:
                new_time = time.time()
                diff = new_time - last_time
                last_time = new_time
                print("batch: {}  loss: {}  speed: {} batches / s".format(
                    i, cst, 100 / diff
                ))

        #saver.save(sess, save_path + "model", write_meta_graph=True)
        #tf.train.write_graph(sess.graph_def, '.', './saved/model_recipe.pbtxt')  
        saver.save(sess, save_path + "model_recipe", write_meta_graph=True)
        #saver.save(sess, save_path + "model", global_step=epoch, write_meta_graph=True)
        #saver.save(sess, save_path + "model")
    

In [ ]:

#output_tensor = "final_outputs"
#output_tensor = "network_output"
#output_tensor = "outputs_reshaped"
#output_tensor = "outputs"
output_tensor = scope+"/outputXYZ"

In [ ]:
def freeze_graph(model_folder, model_file_name = None):
    GPU_USE = '/cpu:0'
    config = tf.ConfigProto(device_count = {"GPU": 0})
    
    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_folder)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_folder = "/".join(input_checkpoint.split('/')[:-1])
    print (absolute_model_folder)
    if (model_file_name == None):
        model_file_name = input_checkpoint.split("/")[-1].split(".")[0]
    else:
        input_checkpoint = absolute_model_folder + "/" + model_file_name
    print (model_file_name)
    
    # If you use our Script outside of the tutorial please specify your path to store the model
    output_graph = absolute_model_folder + "/rnnmodel.pb"

    # Output node for TensorFlow to decide which part of the Graph to keep.

    print("Saver is loading: " + input_checkpoint + '.meta')
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=True)

    # We retrieve the protobuf graph definition
    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()

    # start session and write the exported file. 
    with tf.device(GPU_USE):
        with tf.Session() as sess:
            saver.restore(sess, input_checkpoint)

            output_graph_def = graph_util.convert_variables_to_constants(
                sess, # The session is used to retrieve the weights
                input_graph_def, # The graph_def is used to retrieve the nodes 
                [output_tensor] # The output node names are used to select the useful nodes.
            ) 

            # Serialize and dump the output graph to the filesystem
            with tf.gfile.GFile(output_graph, "wb") as f:
                f.write(output_graph_def.SerializeToString())
            print("%d ops in the final graph." % len(output_graph_def.node))
            print("Wrote model to %s" % output_graph)

In [ ]:
'''graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()
node_names = [node.name for node in input_graph_def.node]
print(node_names)'''


In [ ]:
freeze_graph(save_path)

In [ ]:
'''with tf.gfile.FastGFile("model/rnnmodel.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')'''

In [ ]:
TEST_PREFIX = ""
count =10
with open ('./results1.txt', 'rt') as in_file:
    for cnt,line in enumerate(in_file): 
        #print(line) # prints that line
        tmp = line.split("(")
        #print(tmp)
        if cnt >1 :
            TEST_PREFIX += tmp[0] + ','
        if cnt ==22 :
            TEST_PREFIX += tmp[0]   
        
        if cnt == 22:
            break

In [ ]:
'''absolute_model_folder = "./model"
model_file_name = "model_recipe"
input_checkpoint = absolute_model_folder + "/" + model_file_name'''
#input_checkpoint = "model/model_recipe.meta"

In [ ]:
'''with tf.device(GPU_USE):
    with tf.Session() as sess:
        #saver.restore(sess, input_checkpoint)
        #TEST_PREFIX = "The "
        LEN_TEST_TEXT = 500
        #TEST_PREFIX= "eggs,bread,onion,butter,paprika"
        for i in range(len(TEST_PREFIX)):
            out = net.run_step(embed_to_vocab(TEST_PREFIX[i], vocab), i == 0)
            
        print("Sentence:")
        gen_str = TEST_PREFIX
        for i in range(LEN_TEST_TEXT):
            # Sample character from the network according to the generated
            # output probabilities
            element = np.random.choice(range(len(vocab)), p=out)
            gen_str += vocab[element]
            out = net.run_step(embed_to_vocab(vocab[element], vocab), False)
        print(gen_str)
        f = open('saved/test_recipe.txt','w')
        f.write(TEST_PREFIX + '\n')
        f.write(gen_str)
        f.close()    
    '''
    
    